# Preface
This notebook is a means of documenting why this module behaves the way it does in ways that simply won't 
fit into the actual code of the module. It is not a necessary read to use the module, but if you find that you are 
seeing inconsistent results with what you expected, please read through this to understand the models that are implemented 
and how they are implemented. 

# 1. Implementing a RaschModel

## 1.1: JAG Model Description
An important question anybody has to ask when trying to implement something is what exactly is that something 
that they are trying to implement? In my case, it was: "what exactly is a Rasch model?" When I started this project 
my initial goal was to implement a Cornell paper, *JAG: Joint Assessment Grading*. In this paper, they defined the Rasch model
as 
    
$$ P(X_{ij} = 1 | S_i, Q_j) = \frac{1}{1+e^{-X_{ij} * (S_i - Q_j)}} \tag{1}$$ 

where $X_{ij}$ is an encoded answer, either 1 or 0 on from some
sort of questionnaire. In the case of the paper, it was in the context of an academic test. Thus, $S_i$ was The $i^{th}$ student's 
ability, $Q_J$ the $j^{th}$ question's difficulty, and $X_{ij} = 1 $ represented a correct answer while $X_{ij} = 0 $ represented
an incorrect answer. The estimation of the student ability and question difficulty (according to the JAG paper)
is done through maximizing a likelihood function that is described in the original paper. In practice a negative log likelihood is
minimized which is not defined in the JAG paper.

(Studer) 

## 1.2: Why Explore other descriptions for the Rasch model? 
When I was implementing the Rasch model, I needed to find something to test my implementation against. Initially, following the JAG 
paper, I implemented the Rasch model by maximizing the log likelihood of the model. 

---
### 1.2.1: Side note on Rasch Model Optimization
In reality I was minimizing the negative log likelihood with tensorflow's 
automatic differentiation of the following loss function: 

$$loss = -\sum_{i,j} X_{ij}\log(\sigma(-X_{ij}(S_i-Q_j)) + (1-X_{ij})\log(1-\sigma(-X_{ij}(S_i-Q_j)))$$  

This loss function is common in logistic regression but has no academic citation since the majority of papers exploring the Rasch
model either do not define a loss function for logistic regression in its entirety (A great example of 
this is a paper from SAS on using logistic regression to optimize the Rasch model which 
defines seperate loss functions for the two parameters but does not define how to apply these 
functions in a way to optimize both parameters), or don't use 
logistic regression at all. In the latter case, Marginal Maximum Likelihood or Bayesian estimation techniques are used.
The lack of clarity in the Rasch model's optimization was one of the motivations for 
exploring further model descriptions. 

---

Eventually, I came across a tutorial on pyschometric parameter estimation from Penn State's Social Science Research Institute. 
This tutorial can be found here: https://quantdev.ssri.psu.edu/tutorials/introduction-irt-modeling. What's notable about this 
tutorial is the following lines:

```
"The 1PL (also called the Rasch model) IRT model describes test items in terms of only one parameter, item difficulty, q. Item difficulty is simply how hard an item is (how high does the latent trait ability level need to be in order to have a 50% chance of getting the item right?). q is estimated for each item of the test."
```

The Rasch model this tutorial puts forth is defined as 
$$ P(X_{ij} = 1 | S_i, Q_j) =  \frac{e^{S_i - Q_j}}{1+e^{S_i - Q_j}} \tag{2}$$

Now a inconsistency has been introduced. Thus, investigation into more descriptions needs to occur in an attempt to find which description
can be trusted. 


Note: Equation (2)'s syntax is changed from the tutorial based on my interpretation of the Penn State parameters in terms of the original JAG implementation, 
in particular, $b = Q$ and $\theta = S$. This will remain the same for future equations. 


## 1.3: Chris Hulme-Lowe Dissertation Model Description
While investigating this issue, I came across a dissertation that described IRT parameter estimation and optimization methods in great 
detail. This paper presented the Rasch model as different from the 1PL (1 parameter logistic model), 
"The Rasch model is less restrictive than the 1PL, but more restrictive than the 2PL," (Hulme-Lowe). This is in contrast to 
the Penn State tutorial which presented the Rasch model as equivalent to the 1PL, which can be seen in the previous section's
excerpt. Still, the dissertation presents the Rasch
model nearly identicalally to the Penn State description:

$$ P(X_{ij} = 1 | S_i, Q_j) =  \frac{\exp(Da_j(S_i - Q_j))}{1+\exp(Da_j(S_i - Q_j))} \tag{3} $$

Where, as explained in the paper, D can be thought of as 1. In fact, in most Rasch model software implementations D is set to 1, 
"However, the logistic model has become so ubiquitous in IRT that many modern software packages leave D = 1, 
which produces parameters in the logistic metric," (Hulme-Lowe). As well, $a_j$ is the $j^{th}$ discrimination parameter (a question's ability to determine student ability)
and for the Rasch model $a_j = \bar{a}$ and typically $ \bar{a} = 1 $ 

(However, $\bar{a}$'s value is not discussed by Hulme-Lowe, this information was gathered from
the [R implementation](https://www.rdocumentation.org/packages/ltm/versions/1.1-1/topics/rasch) of the Rasch model, "Although the common formulation of the Rasch model assumes that the discrimination parameter is fixed to 1...").

Thus when simplified the Rasch model as described by Hulme-Lowe is

$$ P(X_{ij} = 1 | S_i, Q_j) =  \frac{\exp(S_i - Q_j)}{1+\exp(S_i - Q_j)} \tag{4} $$

And thus, (4) = (3) = (2), in theory. So from this I can only extrapolate that the Rasch must be equivalent to the 1PL when all of the 
parameters that make the Rasch model distinct are set to 1. The problem being that Penn State claims that only difficulty parameters are estimated
in this case. Yet, the JAG paper asserts that the Rasch model allows the estimation of both student ability and question 
difficulty, "This allows us to estimate the ability of each student and the difficulty of each question by maximizing the
likelihood of all outcomes under our model," (Hulme-Lowe).

---
### 1.3.1: Side Note: Attempting to optimize the Hulme-Lowe 2PL
Hulme-Lowe puts forth multiple methods of optimization in his dissertation. One of them is Marginal Maximum Likelihood estimation.
I attempted to implement this for the 2PL model that he presents (which seems to be closest to the JAG description since it 
estimates student ability and question difficulty) but always had a problem with run away gradients that ended up producing 
a lot of NaN values in the e-step. My work on this implementation
can be found [here](https://gist.github.com/ByrdOfAFeather/56b5fac75d9fee36884f2f9c12ba8ce2). Notably, I set D = 1.702 
since that is how the dissertation uses it (I do not know why). 

The fact that I couldn't implement this in a working fashion was part of the motivation to find other descriptions of the model.

---

## 1.4: A Quick Look At Internet Resources
At this point I was quite confused, as you might imagine, how a paper from Cornell that builds off the Rasch model managed 
to both define the Rasch model so very wrong and get published with that mistake completely uncaught. At least, at this time I was 
under the impression that it was defined incorrectly.
So I decided to do some more investigation. First I started by trying to find an already existing Python implementation of the Rasch model
as described by both the Penn State tutorial and Hulme-Lowe. 
As far as I am aware, this does not exist. I did, however, come across these sources: 

### 1.4.1: A tutorial on how to implement the Rasch model in Tensorflow:
[Source](https://www.kaggle.com/mlarionov/the-rasch-model). This implementation the Rasch model is very close to what
is described by the original JAG paper. However, of note, the multiplication of the exponent by $-X_{ij}$ in the sigmoid 
function is missing. This is a significant factor as the probability that a student got a question correct is always .5 if 
they actually got it wrong when the $-X_{ij}$ is present, which, as far as I am aware that is correct only under the assumption that 
when a student gets a question wrong it's because they guessed with no prior knowledge or ability. What's most confusing about this source is the fact that now we have a ***THIRD*** implementation
of the Rasch model.

i.e. (22.1)

$$ P(X_{ij} = 1 | S_i, Q_j) = \frac{1}{1+e^{(S_i - Q_j)}} $$

However, this is technically equivalent to the Hulme-Lowe when 

$$\bar{a} = \frac{1}{e^{(S_i - Q_j)}}$$

### 1.4.2: David Barber's *Bayesian Reasoning and Machine Learning*
[Source](http://web4.cs.ucl.ac.uk/staff/D.Barber/textbook/090310.pdf).
Where did the previous description of Rasch come from? Well, it seems as credible as any other source. It comes from a book
titled *Bayesian Reasoning and Machine Learning* by David Barber, a professor at the University College of London. Who 
defines the Rasch model (5) and claims that, in fact, both student ability and question difficulty are estimated
by the Rasch model (22.1). 

### 1.4.3: Could Penn State's tutorial be incorrect? 

[Source](https://github.com/lukas-a-olson/item-response-theory/). According to the python notebook of this individual 
also attempting to follow the Penn State tutorial, there is some sort of incorrectness in their visualizations. "This 
clearly indicates an error in the tutorial's visualization: it should peak at 0.25, not 0.5. That lends credibility to 
my analysis, even though I still do not fully understand the model fit." Though, this is only a problem in visualization 
and may or may not reflect on the model. 

Which leaves me with the following 

- JAG's Rasch Model 
- Penn State's Tutorial/Hulme-Lowe's Rasch Model 
- David Barber's Rasch Model

and not a single one of them seems to be more or less correct than the others.

## 1.5: Do the various models produce the "same" results? 

So then, what if all the models are, in fact, correct? After all the Rasch model is simply a function that has been developed 
to behave in a way that provides artificial constraints to student ability based on question difficulty. None of the previously 
defined models really deviate from that, they simply do it on slightly different scales, as far as I am aware. While the 
numeric values are going to be different, perhaps the parameter ordering is the same (i.e. student one has the highest student 
ability even if it varies from 10 to .01). 

The data set is that I'm going to test on is found in the Penn State tutorial. It's an example representing a test
with 500 students and 10 questions. For this we will start with looking at question difficulties, if inconsistencies can
be found there, looking at all 500 students in the test dataset won't be necessary. 


### 1.5.1: David Barber's Implementation (eduTech's Implementation)
For more information on why this is known to be a correct implementation of David Barber's description see the tests/RaschModelTests.ipynb. 

In [1]:
import pandas as pd 
from irt.rasch import RaschModel
def load_test_data():
    with open("tests/ouirt.txt", "r") as f:
        rows = []
        for line in f.readlines():
            current_row = []
            for no in line.split(" "):
                try:
                    current_row.append(int(no))
                except ValueError:
                    continue
            rows.append(current_row)
        return pd.DataFrame(rows)


# The learning rate and epochs are arbitrary and chosen to match the likelihood of 
# the Penn State tutorial, in reality the R implementation use Marginal Maximum likelihood estimation 
# which optimizes the model in a significantly different way. 
test_model = RaschModel(learning_rate=.1)
test_data = load_test_data()
test_model.fit(test_data, epochs=600)
student_abilities, question_difficulties = test_model.get_model_descriptors()
print("======= Question Difficulties =======")
print(question_difficulties)

LOSS AFTER 0 ITERATIONS: -3465.7294921875
LOSS AFTER 100 ITERATIONS: -2812.181396484375
LOSS AFTER 200 ITERATIONS: -2759.679443359375
LOSS AFTER 300 ITERATIONS: -2711.19921875
LOSS AFTER 400 ITERATIONS: -2666.448974609375
LOSS AFTER 500 ITERATIONS: -2625.1435546875
Finished fitting with a final loss of -2587.384765625
======= Question Difficulties =======
[[1.7579474  1.0194758  0.7740919  0.879095   0.2520845  0.7928815
  0.36027655 0.7001337  0.77409184 2.6160252 ]]


### 1.5.2: Penn State/Hulme-Lowe's Implementation

This model is already implemented in R and in the Penn State tutorial, they use the R implementation, thus 
the results are as they are presented in the tutorial.

```
## Model Summary:
##    log.Lik      AIC      BIC
##  -2587.186 5196.372 5242.733
## 
## Coefficients:
##             value std.err  z.vals
## Dffclt.V1  1.6581  0.1303 12.7219
## Dffclt.V2  0.9818  0.1031  9.5233
## Dffclt.V3  0.7499  0.0968  7.7431
## Dffclt.V4  0.8495  0.0993  8.5522
## Dffclt.V5  0.2482  0.0891  2.7851
## Dffclt.V6  0.7677  0.0973  7.8930
## Dffclt.V7  0.3528  0.0900  3.9186
## Dffclt.V8  0.6794  0.0953  7.1311
## Dffclt.V9  0.7499  0.0968  7.7429
## Dffclt.V10 2.3978  0.1764 13.5892
## Dscrmn     1.3782  0.0741 18.5941
```

### 1.5.3: JAG's Implementation

JAG's implementation is a bit more tricky. The model doesn't technically exist already. So a full-proof implementation 
also doesn't quite exist. I will detail what assumptions I made in this implementation and how I implemented it in the hopes 
that if it's correct then it's well documented, or if not, it can be corrected. 

The loss function:

$$loss = \sum_{i,j} X_{ij}\log(\sigma(-X_{ij}(S_i-Q_j)) + (1-X_{ij})\log(1-\sigma(-X_{ij}(S_i-Q_j)))$$  

See 1.2.1 to get some idea on where this comes from, notably, this is the same loss function that is used in the David Baber
implementation aside from the added $-X_{ij}$. 

With the following derivatives:

$$\frac{\partial{loss}}{\partial{S_i}} = \sum_{j} X_{ij} - \sigma(-X_{ij} * (S_i - Q_j))$$

$$\frac{\partial{loss}}{\partial{Q_j}} = -\sum_{i} X_{ij} - \sigma(-X_{ij} * (S_i - Q_j))$$

Thus in Python:

In [7]:
import pandas as pd 
import tensorflow as tf
from irt.rasch import RaschModel

class JAGRachModel(RaschModel):
    def __init__(self, learning_rate):
        RaschModel.__init__(self, learning_rate)
    
    @staticmethod
    def predict(batch_students, batch_questions, results):
        return tf.sigmoid(-results * (batch_students - batch_questions))
    
    def _train(self, calc_loss=False):
        predictions = self.predict(self.student_abilities, self.questions_difficulties, self.results)
        student_ability_gradient = self.learning_rate * self._calc_deriv_student_ability(predictions)
        student_ability_gradient = student_ability_gradient / student_ability_gradient.shape[0]
        
        question_difficulties_gradient = self.learning_rate * self._calc_deriv_question_difficulty(predictions)
        question_difficulties_gradient = question_difficulties_gradient / question_difficulties_gradient.shape[1]
        
        self.student_abilities.assign_add(student_ability_gradient)
        self.questions_difficulties.assign_add(question_difficulties_gradient)
        
        if calc_loss:
            return self._calc_loss(predictions, self.results)
        else:
            return None
    
def load_test_data():
    with open("tests/ouirt.txt", "r") as f:
        rows = []
        for line in f.readlines():
            current_row = []
            for no in line.split(" "):
                try:
                    current_row.append(int(no))
                except ValueError:
                    continue
            rows.append(current_row)
        return pd.DataFrame(rows)


# The learning rate and epochs are arbitrary and chosen to match the likelihood of 
# the Penn State tutorial, in reality the R implementation use Marginal Maximum likelihood estimation 
# which optimizes the model in a significantly different way. 
test_model = JAGRachModel(learning_rate=.001)
test_data = load_test_data()
test_model.fit(test_data, epochs=250)
student_abilities, question_difficulties = test_model.get_model_descriptors()
print("======= Question Difficulties =======")
print(question_difficulties)

LOSS AFTER 0 ITERATIONS: -3465.7294921875
LOSS AFTER 100 ITERATIONS: -2927.439208984375
LOSS AFTER 200 ITERATIONS: -2625.4560546875
Finished fitting with a final loss of -2551.02734375
======= Question Difficulties =======
[[5.040559  3.9364557 3.3865798 3.6371763 1.567964  3.433293  2.0597837
  3.1937835 3.3865442 5.7031174]]


### 1.5.4: Testing the results

In [8]:
import numpy as np
question_difficulties_david_barber = [1.7579474, 1.019475, 0.7740919, 0.879095, 0.2520845, 0.7928815, 
                                      0.36027655, 0.7001337, 0.77409184, 2.6160252 ]
question_difficulties_penn_state = [1.6581, .9818, .7499, .8495, .2482, .7677, .3528, .6794, .7499, 2.3978]
question_difficulties_JAG = [5.040559, 3.9364557, 3.3865798, 3.6371763, 1.567964, 3.433293, 2.0597837, 3.1937835, 
                             3.3865442, 5.7031174]


sorted_indexes_question_david_barber = np.argsort(question_difficulties_david_barber)
sorted_indexes_question_penn_state = np.argsort(question_difficulties_penn_state)
sorted_indexes_difficulties_JAG = np.argsort(question_difficulties_JAG)
for current_question_index, index_david_barber in enumerate(sorted_indexes_question_david_barber):
    if sorted_indexes_question_penn_state[current_question_index] == 2 or sorted_indexes_question_penn_state[current_question_index] == 8:
        continue
    
    assert index_david_barber == sorted_indexes_difficulties_JAG[current_question_index] \
           and index_david_barber == sorted_indexes_question_penn_state[current_question_index], \
        f"ERROR AT QUESTION {current_question_index}\n" \
        f"Barber thinks that the current index should be {index_david_barber}\n" \
        f"Penn State thinks that the current index should be {sorted_indexes_question_penn_state[current_question_index]}\n" \
        f"JAG thinks that the current index should be {sorted_indexes_difficulties_JAG[current_question_index]}" 

print("Value's order checks out!")

Value's order checks out!


### 1.5.5: Results

One thing to note here is that the Penn State tutorial ended up having a model with a repeating question difficulty. This
makes these points harder to compare, but the indices are the same as the other sorted lists
same if the indices with the same value are swapped (8 with 2 and 2 with 8). Thus question difficulties are of the same 
meaning. We can infer from all models that Question 10 is a harder question than any other question and question 5 is easier than any other question, 
indicative that all models are in fact equivalent. 

There's still one caveat. ***Penn State doesn't have student abilities because of its assertion that the 
Rasch model does not estimate student abilities***. Even though the David Barber's description does optimize the 
student abilities and the resulting question difficulties are incredibly similar. 

## 1.6: Conclusion

What is correct? The models are produce something that can be argued to be equivalent but why are they different in the 
first place? For optimization purposes? For different constraints? It's unclear. JAG and Berber both use logistic regression
while Hulme-Lowe and Penn State both use Marginal Maximum Likelihood, however, as mentioned in 1.2.1, there's a paper 
that focuses entirely on logistic regression from SAS that defined the Rasch model in the same way that Hulme-Lowe does (Pan). 
Except in that paper, both student ability and question difficulty are estimated from that model (Pan).

It seems as though there's a multitude of ways to actually define the Rasch model, each having a semantic equivalence but 
numeric difference. For eduTech's implementation, I've decided to implement the Berber description for the following 
reasons: 

- It's simple with clearly defined derivatives
- It is the closest to the Penn State tutorial in terms of question difficulties, but also estimates student abilities
- Compared to Marginal Maximum Likelihood estimation it's significantly less computationally expensive 

Future plans for the item response theory module of eduTech includes the 3PL and choosing methods of optimization. Other 
models could be implemented but I no longer know what the difference between the Berber-Rasch model and the 2PL is.

About:

Version: 1.0

Last Updated: 10/22/19

---

# BIBLIOGRAPHY 

“22.1 The Rasch Model.” Bayesian Reasoning and Machine Learning, by David Barber, University Cambridge Press, 2012, pp. 403–404.

Hulme-Lowe, Chris. “Regularized Marginal Maximum Likelihood: The Use of Shrinkage and Selection Operators for Item Parameter Estimation in the Two-Parameter Logistic Model.”

Mlarionov. “The Rasch Model.” Kaggle, Kaggle, 21 Oct. 2018, www.kaggle.com/mlarionov/the-rasch-model.

Pan, Tianshu. “Fitting the Rasch Model under the Logistic Regression Framework to Reduce Estimation Bias.” 
Journal of Modern Applied Statistical Methods, vol. 17, no. 1, 2018, doi:10.22237/jmasm/1530028025.

Studer, Christoph, and Igor Labutov. “JAG: A Crowdsourcing Framework for Joint Assessment and Peer Grading.” AAAI Publications, Thirty-First AAAI Conference on Artificial Intelligence, 12 Feb. 2017.

Wood, Julie, and Peter v C.M. Molenaar. “Introduction to IRT Modeling.” Quantdev, 12 Nov. 2017, quantdev.ssri.psu.edu/tutorials/introduction-irt-modeling.